In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font',family='Malgun Gothic')
import seaborn as sns
import pandas as pd

#### x 범주형, y 범주형 : 주로 독립성 증명
- 선정 변수 : type_of_contract, cancellation
- 분석 내용: 계약 유형에 따라 해약 여부가 달라지는가? 
    - 귀무가설: 계약유형에 따라 해약여부가 달라지지 않는다. 
    - 대립가설: 계약유형에 따라 해약여부가 달라진다. 
- 분석 결론 (기준 : p-value 0.05)
    - 통계 결론: 대립가설이 참 (귀무가설 기각)
    - 사실 결론: 계약유형에 따라 해약여부가 달라진다. 

In [2]:
df_TOCC = pd.read_csv('../../../datasets/TypeOfContractChannel.csv')
df_TOCC[:2]

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0


In [9]:
df_TOCC[['type_of_contract', 'cancellation']].describe()
#항목 갯수 확인 

,type_of_contract,cancellation
count,51300,51279
unique,2,2
top,렌탈,정상
freq,46481,50372


In [10]:
df_TOCC_category = df_TOCC[['type_of_contract', 'cancellation']]

In [12]:
df_TOCC_category_pivot =pd.pivot_table(data=df_TOCC_category, index='type_of_contract', columns='cancellation', aggfunc=len)
#목표변수가 groupby의 대상 -> index 
#집합함수 aggfunc -> count되니까 len 
df_TOCC_category_pivot

cancellation,정상,해약
type_of_contract,,
렌탈,46137,344
멤버십,4231,563


In [15]:
import scipy
scipy.__version__
import scipy.stats as stats 

In [18]:
stats.chi2_contingency(observed=df_TOCC_category_pivot, correction=True)
#값의 독립성 확인 

#correction은 자유도(degree of freedom). 모집단의 것을 샘플로 가져왔다는 걸 인정. 샘플에서는 자유도 -1을 주로함(실험제외). 
#correction=true면 1 #return에 t-test 평균을 중심으로 한 분산 수치값, p-value, degree of freedom 

#건수에 큰 차이가 있기 때문에 두가지 다 정규분포(확률적 분포)로 끌고 오면 비교 가능
#분포로 변환을 시켜서 확률적으로 끌어와서 p-value 확인 


Chi2ContingencyResult(statistic=3021.9735309482894, pvalue=0.0, dof=1, expected_freq=array([[45658.8007411,   822.1992589],
       [ 4709.1992589,    84.8007411]]))

#### 선정 변수 : product, cancellation
- 분석 내용: 차종에 따라 해약 여부가 달라지는가? 
    - 귀무가설: 차종에 따라 해약여부가 달라지지 않는다. 
    - 대립가설: 차종에 따라 해약여부가 달라진다. 
- 분석 결론 (기준 : p-value 0.05)
    - 통계 결론: 대립가설 
    - 사실 결론: 차종에 따라 해약여부가 달라진다. 

In [24]:
# 다른 변수 이름 작성 시 copy() 사용 
# df_TOCC_product = df_TOCC[['product', 'cancellation']].copy()
# df_TOCC_product[:2]

# copy를 안쓰면 똑같은 영역을 공유하게 되어서 한쪽을 바꾸면 다른 한쪽도 같이 바뀜. copy를 사용해야 별개의 공간 사용.
# df_TOCC = df_TOCC[['product', 'cancellation']]덮어쓸떄는 안써도 됨. 새로운 변수지정할때는 써야함. 

,product,cancellation
0,K1,정상
1,K1,정상


In [26]:
# df_TOCC_product.describe()
# count수 차이가 있다는 것은 한쪽에 null이 있고 이를 처리해야함을 할 수 있다 
# unique보고 범주형임을 확인 

,product,cancellation
count,51303,51279
unique,6,2
top,K1,정상
freq,39134,50372


In [49]:
df_TOCC_product = df_TOCC[['product', 'cancellation']].dropna().copy()
df_TOCC_product.describe()

,product,cancellation
count,51278,51278
unique,6,2
top,K1,정상
freq,39111,50371


In [33]:
df_TOCC_product_pivot = pd.pivot_table(data=df_TOCC_product
              , index='product', columns='cancellation', aggfunc=len)
# groupby대상을 index에 넣기 
# 이건 function사용해서 copy사용필요 없음 

In [35]:
# product 기준 
stats.chi2_contingency(observed=df_TOCC_product_pivot, correction=True)
# p-value가 0.05보다 작아서 관계가 있다 

Chi2ContingencyResult(statistic=103.79474317337241, pvalue=8.372028115481391e-21, dof=5, expected_freq=array([[3.84192086e+04, 6.91791353e+02],
       [8.83393274e+03, 1.59067261e+02],
       [2.04517380e+03, 3.68262023e+01],
       [3.21216058e+02, 5.78394243e+00],
       [6.33591306e+02, 1.14086938e+01],
       [1.17877452e+02, 2.12254768e+00]]))

In [50]:
df_TOCC_product_pivot = pd.pivot_table(data=df_TOCC_product
              , index='cancellation', columns='product', aggfunc=len)

In [55]:
# cancellation 기준 
stats.chi2_contingency(observed=df_TOCC_product_pivot, correction=True)
# index와 columns가 바뀌어도 값은 똑같이 나온다 

Chi2ContingencyResult(statistic=103.79474317337241, pvalue=8.372028115481391e-21, dof=5, expected_freq=array([[3.84192086e+04, 8.83393274e+03, 2.04517380e+03, 3.21216058e+02,
        6.33591306e+02, 1.17877452e+02],
       [6.91791353e+02, 1.59067261e+02, 3.68262023e+01, 5.78394243e+00,
        1.14086938e+01, 2.12254768e+00]]))

#### 선정 변수 : state, cancellation (카이 제곱할 수 없는 상황)
- 이런 변수 구성은 참조되지 않는다. 

In [41]:
#error with Nan
# df_TOCC_error = df_TOCC[['state', 'cancellation']]
# #df_TOCC_error[:2]
# df_TOCC_error.describe()

,state,cancellation
count,51304,51279
unique,4,2
top,계약확정,정상
freq,50350,50372


In [42]:
df_TOCC_error = df_TOCC[['state', 'cancellation']].dropna().copy()
df_TOCC_error.describe()

,state,cancellation
count,51279,51279
unique,4,2
top,계약확정,정상
freq,50326,50372


In [46]:
df_TOCC_error_pivot = pd.pivot_table(data=df_TOCC_error
              , index='state', columns='cancellation',
              aggfunc=len)
df_TOCC_error_pivot
# 레코드에 null이 없지만, 이렇게 pivot table에서 두가지 상황을 조합했을때 null이 있는 경우는 카이제곱을 사용하지 못한다.
# p-value 구할 수 없다 
# 이럴때는 귀무가설이 맞게 됨. 사용할 수 없는 상황이다. 


cancellation,정상,해약
state,,
계약확정,50326.0,NaN
기간만료,32.0,NaN
해약진행중,14.0,NaN
해약확정,NaN,907.0


In [47]:
stats.chi2_contingency(observed=df_TOCC_error_pivot
                       , correction=True)

Chi2ContingencyResult(statistic=nan, pvalue=nan, dof=3, expected_freq=array([[nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan]]))